# 1. A influência da aleatoriedade  na validação do modelo

In [2]:
import pandas as pd
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [4]:
dados.head(2)

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362


In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco","idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x,teste_x,treino_y,teste_y = train_test_split(x,y,test_size = 0.25, stratify = y)

print(f"Treinamos o modelo com {len(treino_x)} e testaremos com {len(teste_x)}")

Treinamos o modelo com 7500 e testaremos com 2500


In [17]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x,treino_y)
acuracia= dummy_stratified.score(teste_x,teste_y)*100

print(f"A acurácia do dummy stratified foi de {acuracia:0.2f}%")

A acurácia do dummy stratified foi de 58.00%


In [22]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier()
modelo.fit(treino_x,treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y,previsoes)*100
#acuracia = modelo.score(teste_x,teste_y)*100
print(f"A acurácia do modelo foi de {acuracia:0.2f}%")

A acurácia do modelo foi de 69.60%


In [23]:
SEED = 5
np.random.seed(SEED)
treino_x,teste_x,treino_y,teste_y = train_test_split(x,y,test_size = 0.25, stratify = y)

print(f"Treinamos o modelo com {len(treino_x)} e testaremos com {len(teste_x)}")

modelo = DecisionTreeClassifier()
modelo.fit(treino_x,treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y,previsoes)*100
#acuracia = modelo.score(teste_x,teste_y)*100
print(f"A acurácia do modelo foi de {acuracia:0.2f}%")

Treinamos o modelo com 7500 e testaremos com 2500
A acurácia do modelo foi de 73.32%


## 1.2 Validação cruzada

In [36]:
from sklearn.model_selection import cross_validate

SEED = 57
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x , y , cv = 5, return_train_score = False)
media = results["test_score"].mean()
desvio_padrao = results["test_score"].std()
print(f"Accuracy com cross validation 5 =  [{(media - 2*desvio_padrao)*100:0.2f},{(media + 2*desvio_padrao)*100:0.2f}]")

Accuracy com cross validation 5 =  [75.21,76.35]


# 2. Kfold com aleatorização

Aleatoriedade no cross validate

In [39]:
from sklearn.model_selection import KFold

SEED = 57
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x , y , cv = cv, return_train_score = False)
imprime_resultado(results)

Accuracy com cross validation 5 =  [74.37,77.19]


In [79]:
def imprime_resultado(results):
  media = results["test_score"].mean()
  desvio_padrao = results["test_score"].std()
  print(f"Accuracy média: {media*100:0.2f}%")
  print(f"Accuracy intervalo: [{(media - 2*desvio_padrao)*100:0.2f},{(media + 2*desvio_padrao)*100:0.2f}]")

In [46]:
from sklearn.model_selection import KFold

SEED = 57
np.random.seed(SEED)

cv = KFold(n_splits = 10,shuffle=True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x , y , cv = cv, return_train_score = False)
imprime_resultado(results)

Accuracy média: 75.78%
Accuracy intervalo: =  [73.77,77.79]


# 3. Estratificação

## 3.1 Simulando situação horrível de azar (imbalance)

In [48]:
dados_com_azar = dados.sort_values("vendido", ascending = True)
x_azar = dados_com_azar[["preco","idade_do_modelo","km_por_ano"]]
y_azar = dados_com_azar["vendido"]
dados_com_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


Sem o shuffle

In [49]:
from sklearn.model_selection import KFold

SEED = 57
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar , y_azar , cv = cv, return_train_score = False)
imprime_resultado(results)

Accuracy média: 57.84%
Accuracy intervalo: =  [34.29,81.39]


Com o shuffle

In [51]:
from sklearn.model_selection import KFold

SEED = 57
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar , y_azar , cv = cv, return_train_score = False)
imprime_resultado(results)

Accuracy média: 75.78%
Accuracy intervalo: =  [73.54,78.02]


USANDO O STRATIFIED KFOLD

In [55]:
from sklearn.model_selection import StratifiedKFold

SEED = 57
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar , y_azar , cv = cv, return_train_score = False)
imprime_resultado(results)

Accuracy média: 75.76%
Accuracy intervalo: =  [73.78,77.74]


# 4. Dados Agrupáveis

## 4.1 Gerando dados aleatórios

In [59]:
np.random.seed(SEED)
dados["modelo_aleatorio"] = dados.idade_do_modelo + np.random.randint(-2,3,size = 10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,10
3,95276.14,0,3,43167.32682,1
4,117384.68,1,4,12770.11290,3


In [60]:
dados.modelo_aleatorio.unique()

array([16, 20, 10,  1,  3, 14, 11, 17,  2,  9, 12, 18,  7,  6, 15, 13, 19,
       21,  5,  4,  8, 22,  0, -1])

In [61]:
dados.modelo_aleatorio.min()

-1

In [62]:
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1

In [63]:
dados.modelo_aleatorio.unique()

array([18, 22, 12,  3,  5, 16, 13, 19,  4, 11, 14, 20,  9,  8, 17, 15, 21,
       23,  7,  6, 10, 24,  2,  1])

In [64]:
dados["modelo"] = dados.modelo_aleatorio

In [67]:
dados.drop("modelo_aleatorio",axis=1,inplace = True)

In [68]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,3
4,117384.68,1,4,12770.11290,5


## 4.2 Agrupamento por GroupKFold

In [70]:
from sklearn.model_selection import GroupKFold

SEED = 57
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar , y_azar , cv = cv, groups = dados["modelo"], return_train_score = False)
imprime_resultado(results)

Accuracy média: 75.78%
Accuracy intervalo: =  [73.30,78.27]


# 5. Pipeline de treino e validação

## 5.1 Cross Validation com StandarScaler

In [73]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado,treino_y)
acuracia = modelo.score(teste_x_escalado, teste_y)*100

print(f"A acurácia foi de {acuracia}%")


A acurácia foi de 77.48%


In [76]:
from sklearn.model_selection import GroupKFold

SEED = 57
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar_escalado , y_azar , cv = cv, groups = dados["modelo"], return_train_score = False)
imprime_resultado(results)

Accuracy média: 76.69%
Accuracy intervalo: =  [74.53,78.84]


In [75]:
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)

In [80]:
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([("tranformacao",scaler),("estimador",modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, x_azar , y_azar , cv = cv, groups = dados["modelo"], return_train_score = False)
imprime_resultado(results)

Accuracy média: 76.69%
Accuracy intervalo: [74.49,78.88]
